In [85]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math
import requests as r
import re
import numpy as np
from bs4 import BeautifulSoup as bs
import warnings
warnings.filterwarnings('ignore')

# Лабиринт

In [170]:
def authorsDict(data):
    author_rows = data.find_all("div", {"class": "authors"})
    authors_dict = {}
    for row in author_rows:
        text = row.text.split(': ')
        authors_dict[text[0]] = text[1]
    return authors_dict

In [389]:
def getBookInfo(bookID):
#     s = 'https://www.labirint.ru/books/622004/'
#     s = 'https://www.labirint.ru/books/710102/'
#     s = 'https://www.labirint.ru/books/845530'
    url = 'https://www.labirint.ru/books/' + str(bookID)
    resp = r.get(url, verify=False)
    if resp.status_code == 200:
        soup_data = bs(resp.text)
        authors_dict = authorsDict(soup_data)
        if 'Автор' in authors_dict:
            author_value = authors_dict['Автор']
        else:
            author_value = ''
        if 'Переводчик' in authors_dict:
            interpreter_value = authors_dict['Переводчик']
        else:
            interpreter_value = ''
        if 'Редактор' in authors_dict:
            editor_value = authors_dict['Редактор']
        else:
            editor_value = ''
        book_name = soup_data.find("div", {"class": "prodtitle", "id": "product-title"})
        publisher = soup_data.find("div", {"class": "publisher"})
        series = soup_data.find("div", {"class": "series"})
        genre = soup_data.find("div", {"class": "genre"})
        cycle = soup_data.find("div", {"class": "cycle"})
        not_selling = soup_data.find_all("div", {"class": "buying-price missing-price"})
        without_discount = soup_data.find("div", {"class": "buying-price"})
        if publisher:
            publisher_value = publisher.text.split(': ')[1].split(', ')[0]
            year_published = publisher.text.split(': ')[1].split(', ')[1]
        else:
            publisher_value = ''
            year_published = ''
        if book_name:
            book_name_value = book_name.h1.text.split(': ')[1]
        else:
            book_name = ''
        if series:
            series_value = soup_data.find("div", {"class": "series"}).text.split(': ')[1]
        else:
            series_value = ''
        if genre:
            genre_value = soup_data.find("div", {"class": "genre"}).text.split(': ')[1]
        else:
            genre_value = ''
        if cycle:
            cycle_value = soup_data.find("div", {"class": "cycle"}).text.split(': ')[1]
        else:
            cycle_value = ''
        if not_selling:
            availibility_value = 'Нет в наличии'
            old_price_value = np.nan
            new_price_value = np.nan
        else:
            availibility_value = 'Есть в наличии'
            if without_discount:
                old_price_value = int(re.sub(
                    r'[^0-9]',
                    '',
                    re.findall(r'(.*)\n', soup_data.find("div", {"class": "buying-price"}).text)[2]
                ))
                new_price_value = np.nan
            else:
                old_price_value = int(soup_data.find("div", {"class": "buying-priceold-val"}).text)
                new_price_value = int(re.search(r'\n(.*)\n', soup_data.find("div", {"class": "buying-pricenew-val"}).text).group(1))
        
        column_names = [
            'Наличие',
            'Наименование',
            'Автор',
            'Переводчик', 
            'Редактор',
            'Издательство',
            'Год издания',
            'Серия',
            'Цикл',
            'Жанр',
            'Цена без скидки',
            'Цена со скидкой',
            'Ссылка на товар'
        ]
        
        resulting_df = pd.DataFrame([[
            availibility_value,
            book_name_value,
            author_value,
            interpreter_value,
            editor_value,
            publisher_value,
            year_published,
            series_value, 
            cycle_value, 
            genre_value, 
            old_price_value, 
            new_price_value,
            url
        ]
        ], columns=column_names)
        return resulting_df
    else:
        return None

In [391]:
def labirintSearch(name):
    url = 'https://www.labirint.ru/search/' + name
    resp = r.get(url)
    if resp.status_code == 200:
        soup_data = bs(resp.text)
        results_count_row = soup_data.find("a", {
                "class": "b-stab-e-slider-item-e-txt js-search-result-tab", 
                "data-id_tab": "0"
            })
        if results_count_row:
            results_count_value = int(re.sub(r'[^0-9]', '', results_count_row.text))
            books_result = []
            for i in range(results_count_value//60+1):
                inner_url = 'https://www.labirint.ru/search/' + name + '/?stype=0&page=' + str(i+1)
                inner_resp = r.get(inner_url)
                if inner_resp.status_code == 200:
                    inner_soup_data = bs(inner_resp.text)
                    inner_search_results = inner_soup_data.find_all("a", {"class": "product-title-link"})
                    if inner_search_results:
                        for book in inner_search_results:
                            books_result.append(re.sub(r'[^0-9]', '', book.attrs['href']))
        else:
            books_result = []
        return books_result
        

In [395]:
def allResultsDF(name):
    books_list = labirintSearch(name)
    books_df = pd.DataFrame()
    for book in books_list:
        book_df = getBookInfo(book)
        books_df = pd.concat([books_df, book_df])
    return books_df

In [418]:
def cheapestResultDF(name):
#     books_df = allResultsDF(name)
    books_df = res_df
    cols_to_group = ['Наименование', 'Автор']
    books_df['min_price'] = books_df[['Цена без скидки', 'Цена со скидкой']].min(axis=1)
#     return books_df
    grouped_books_df = books_df.groupby(cols_to_group).min('min_price')
    return grouped_books_df

In [390]:
getBookInfo(622004)

,Наличие,Наименование,Автор,Переводчик,Редактор,Издательство,Год издания,Серия,Цикл,Жанр,Цена без скидки,Цена со скидкой,Ссылка на товар
0,Есть в наличии,Темный лес,Лю Цысинь,Накамура Дмитрий,Горинова Н.,Fanzon,2017 г.,Sci-Fi Universe. Лучшая новая НФ,Воспоминания о прошлом Земли. 3 книги,Современная зарубежная проза,1122,819,https://www.labirint.ru/books/622004


In [392]:
test_res = labirintSearch('Лю Цысинь')

In [393]:
len(test_res)

9

In [394]:
test_res

['622004',
 '830130',
 '610958',
 '800577',
 '720633',
 '701229',
 '653704',
 '766357',
 '752231']

In [415]:
%%time
res_df = resultsDF('эрих мария ремарк')

CPU times: user 59.2 s, sys: 835 ms, total: 1min
Wall time: 7min 30s


In [416]:
res_df

,Наличие,Наименование,Автор,Переводчик,Редактор,Издательство,Год издания,Серия,Цикл,Жанр,Цена без скидки,Цена со скидкой,Ссылка на товар
0,Есть в наличии,На западном фронте без перемен,Ремарк Эрих Мария,Афонькин Ю.,,АСТ,2021 г.,Шедевры мировой литературы,,,844.0,616.0,https://www.labirint.ru/books/813041
0,Есть в наличии,Последняя остановка,Ремарк Эрих Мария,Зись Елена,,АСТ,2021 г.,Возвращение с Западного фронта,,,748.0,546.0,https://www.labirint.ru/books/453691
0,Есть в наличии,"""Скажи мне, что ты меня любишь...""",Ремарк Эрих Мария,"Факторович Е. П., Бабенко В.",,АСТ,2022 г.,Эксклюзивная классика,,,327.0,NaN,https://www.labirint.ru/books/843742
0,Есть в наличии,Три товарища,Ремарк Эрих Мария,Архипов Юрий,,Corpus,2015 г.,Эксклюзивная классика с иллюстрациями,,,3898.0,2846.0,https://www.labirint.ru/books/496325
0,Есть в наличии,Der Funke Leben,Ремарк Эрих Мария,,,Каро,2015 г.,Чтение в оригинале.Немецкий язык,,,1027.0,514.0,https://www.labirint.ru/books/334161
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Нет в наличии,Тени в раю (2CDmp3),Ремарк Эрих Мария,"Котелкин В., Черная Л.",,Аудиокнига,2015 г.,,,,NaN,NaN,https://www.labirint.ru/books/515344
0,Нет в наличии,Жизнь взаймы (CDmp3),Ремарк Эрих Мария,,,Ардис,2009 г.,Зарубежная проза XX век,,,NaN,NaN,https://www.labirint.ru/books/194408
0,Нет в наличии,Три товарища (CDmp3),Ремарк Эрих Мария,,,ИДДК,2006 г.,Аудиокнига,,,NaN,NaN,https://www.labirint.ru/books/119681
0,Нет в наличии,Три товарища,Ремарк Эрих Мария,,,Эксмо-Пресс,2002 г.,,,,NaN,NaN,https://www.labirint.ru/books/9327


In [420]:
min_test_df = cheapestResultDF('')

In [422]:
pd.set_option('display.max_rows', 1000)

pd.set_option('display.max_columns', 100)

In [423]:
min_test_df

,,Цена без скидки,Цена со скидкой,min_price
Наименование,Автор,,,
"""Скажи мне, что ты меня любишь...""",Ремарк Эрих Мария,327.0,NaN,327.0
"""Скажи мне, что ты меня любишь..."" Письма к Марлен Дитрих",Ремарк Эрих Мария,NaN,NaN,NaN
"""Скажи мне, что ты меня любишь..."". Роман в письмах","Ремарк Эрих Мария, Дитрих Марлен",NaN,NaN,NaN
All Quiet on the Western Front,Ремарк Эрих Мария,NaN,NaN,NaN
Als waere alles das letzte Mal,Ремарк Эрих Мария,NaN,NaN,NaN
Arc de Triomphe,Ремарк Эрих Мария,547.0,NaN,547.0
Das gelobte Land,Ремарк Эрих Мария,3282.0,1641.0,1641.0
Der Feind,Ремарк Эрих Мария,NaN,NaN,NaN
Der Funke Leben,Ремарк Эрих Мария,1027.0,514.0,514.0


# ЧГ

In [ ]:
https://www.chitai-gorod.ru/catalog/book/1007290/

In [447]:
def propertiesDict(data):
    properties = data.find_all("div", {"class": "product-prop"})
    properties_dict = {}
    for prop in properties:
        prop_name = prop.find("div", {"class": "product-prop__title"}).text
        prop_value = re.sub(
            '\s{2,}',
            '',
            re.sub(
                '[\n]',
                '',
                prop.find("div", {"class": "product-prop__value"}).text
            )
        )
        properties_dict[prop_name] = prop_value
    return properties_dict

In [449]:
propertiesDict(soup_dt)

{'ID товара': '2611714',
 'Цикл': 'В память о прошлом Земли',
 'Том': '1',
 'Издательство': 'Fanzon',
 'Серия': 'Sci-Fi Universe. Лучшая новая НФ',
 'Год издания': '2020',
 'ISBN': '978-5-04-089112-2Скрыть',
 'Кол-во страниц': '464',
 'Формат': '21.9 x 14.5 x 2.7',
 'Тип обложки': 'Суперобложка',
 'Тираж': '5000',
 'Вес, г': '540',
 'Возрастные ограничения': '16+'}

In [ ]:
def getBookInfo(bookID):
#     s = 'https://www.labirint.ru/books/622004/'
#     s = 'https://www.labirint.ru/books/710102/'
#     s = 'https://www.labirint.ru/books/845530'
    url = 'https://www.chitai-gorod.ru/catalog/book/' + str(bookID)
    resp = r.get(url, verify=False)
    if resp.status_code == 200:
        soup_data = bs(resp.text)
        author = soup_data.find("div", {"class": "product__header"}).find("a", {"class": "link product__author"})
        book_name = soup_data.find("div", {"class": "product__header"}).find("h1", {"class": "product__title js-analytic-product-title"})
        book_properties = soup_data.find_all("div", {"class": "product-prop"})
        properties_dict = propertiesDict(book_properties)
        publisher = properties_dict['Издательство']
        series = properties_dict['Серия']
        genre = ''
        cycle = properties_dict['Цикл']
        not_selling = 
        without_discount = 
        
        if author:
            author_value = re.sub(
                '\s{2,}',
                '',
                re.sub(
                    '[\n]',
                    '',
                    author.text
                )
            )
        else:
            author_value = ''
        
        if book_name:
            book_name_value = re.sub(
                '\s{2,}',
                '',
                re.sub(
                    '[\n]',
                    '',
                    book_name.text
                )
            )
        else:
            book_name_value = ''
            
        if publisher:
            publisher_value = publisher.text.split(': ')[1].split(', ')[0]
            year_published = publisher.text.split(': ')[1].split(', ')[1]
        else:
            publisher_value = ''
            year_published = ''
        if book_name:
            book_name_value = book_name.h1.text.split(': ')[1]
        else:
            book_name = ''
        if series:
            series_value = soup_data.find("div", {"class": "series"}).text.split(': ')[1]
        else:
            series_value = ''
        if genre:
            genre_value = soup_data.find("div", {"class": "genre"}).text.split(': ')[1]
        else:
            genre_value = ''
        if cycle:
            cycle_value = soup_data.find("div", {"class": "cycle"}).text.split(': ')[1]
        else:
            cycle_value = ''
        if not_selling:
            availibility_value = 'Нет в наличии'
            old_price_value = np.nan
            new_price_value = np.nan
        else:
            availibility_value = 'Есть в наличии'
            if without_discount:
                old_price_value = int(re.sub(
                    r'[^0-9]',
                    '',
                    re.findall(r'(.*)\n', soup_data.find("div", {"class": "buying-price"}).text)[2]
                ))
                new_price_value = np.nan
            else:
                old_price_value = int(soup_data.find("div", {"class": "buying-priceold-val"}).text)
                new_price_value = int(re.search(r'\n(.*)\n', soup_data.find("div", {"class": "buying-pricenew-val"}).text).group(1))
        
        column_names = [
            'Наличие',
            'Наименование',
            'Автор',
            'Переводчик', 
            'Редактор',
            'Издательство',
            'Год издания',
            'Серия',
            'Цикл',
            'Жанр',
            'Цена без скидки',
            'Цена со скидкой',
            'Ссылка на товар'
        ]
        
        resulting_df = pd.DataFrame([[
            availibility_value,
            book_name_value,
            author_value,
            interpreter_value,
            editor_value,
            publisher_value,
            year_published,
            series_value, 
            cycle_value, 
            genre_value, 
            old_price_value, 
            new_price_value,
            url
        ]
        ], columns=column_names)
        return resulting_df
    else:
        return None

In [424]:
s = 'https://www.chitai-gorod.ru/catalog/book/1007290/'
resp = r.get(s, verify=False)
soup_dt = bs(resp.text)

In [434]:
soup_dt.find("div", {"class": "product__header"}).find("h1", {"class": "product__title js-analytic-product-title"}).text

'\n                    Задача трех тел\n                '

In [448]:
propertiesDict(soup_dt)

{'ID товара': '2611714',
 'Цикл': 'В память о прошлом Земли',
 'Том': '1',
 'Издательство': 'Fanzon',
 'Серия': 'Sci-Fi Universe. Лучшая новая НФ',
 'Год издания': '2020',
 'ISBN': '978-5-04-089112-2Скрыть',
 'Кол-во страниц': '464',
 'Формат': '21.9 x 14.5 x 2.7',
 'Тип обложки': 'Суперобложка',
 'Тираж': '5000',
 'Вес, г': '540',
 'Возрастные ограничения': '16+'}

In [444]:
soup_dt.find_all("div", {"class": "product-prop"})[1].find
# div class="product-prop__title"

[<div class="product-prop">
 <div class="product-prop__title">ID товара</div>
 <div class="product-prop__value">
                                         2611714
                                     </div>
 </div>,
 <div class="product-prop">
 <div class="product-prop__title">Цикл</div>
 <div class="product-prop__value">
 <a class="link" href="/books-series/v_pamyat_o_proshlom_zemli/242/">
                                                                                     В память о прошлом Земли
                                                                             </a>
 </div>
 </div>,
 <div class="product-prop">
 <div class="product-prop__title">Том</div>
 <div class="product-prop__value">
                                         1
                                     </div>
 </div>,
 <div class="product-prop">
 <div class="product-prop__title">Издательство</div>
 <div class="product-prop__value">
 <a class="link" href="/books/publishers/fanzon/">
 <span itemprop="publisher">

In [435]:
re.sub(
    '\s{2,}',
    '',
    re.sub(
        '[\n]',
        '',
        soup_dt.find("div", {"class": "product__header"}).find("h1", {"class": "product__title js-analytic-product-title"}).text
    )
)

'Задача трех тел'

In [433]:
re.sub(
    '\s{2,}',
    '',
    re.sub(
        '[\n]',
        '',
        soup_dt.find("div", {"class": "product__header"}).find("a", {"class": "link product__author"}).text
    )
)

'Лю Цысинь'

In [ ]:
<div class="product__header">

                <h1 class="product__title js-analytic-product-title" itemprop="name">
                    Задача трех тел
                </h1>

                                    <span class="hidden" itemscope="itemscope" itemtype="https://schema.org/Person" itemprop="author">
                        <meta itemprop="name" content="Лю Цысинь">
                    </span>
                    <a class="link product__author" href="/books/authors/lyu_tsysin/">
                        Лю Цысинь
                    </a>
                
                            </div>

# Testing

In [278]:
# s = 'https://www.labirint.ru/books/622004/'
# s = 'https://www.labirint.ru/books/710102/'
s = 'https://www.labirint.ru/books/845530/'
# s = 'https://www.labirint.ru/search/%2Ctueobq%20pf%20dtnhjv/?stype=0'
# s = 'https://www.labirint.ru/search/эрих%20мария%20ремарк/?stype=0&page=2'
resp = r.get(s, verify=False)
soup_dt = bs(resp.text)

In [284]:
soup_dt.find("div", {
    "class": "prodtitle", 
    "id": "product-title"
}).h1.text.split(': ')[1]
# '\nЭрих Ремарк: Возлюби ближнего своего\n\n\n\nЕсть другие издания\n\n\nLiebe deinen nachsten\n'


'Погоня за панкерой'

In [ ]:
<div id="product-title" class="prodtitle">
<h1>Роберт Хайнлайн: Погоня за панкерой</h1><div class="prodtitle-availibility rang-available"><span>На складе</span></div>					
<div class="cleaner0"></div>
<h2 class="h2_eng">A PURSUIT OF THE PANKERA</h2>
</div>

In [248]:
(soup_dt.find("a", {
    "class": "b-stab-e-slider-item-e-txt js-search-result-tab", 
    "data-id_tab": "0"
}))

bs4.element.Tag

In [250]:
if (soup_dt.find("a", {
    "class": "b-stab-e-slider-item-e-txt js-search-result-tab", 
    "data-id_tab": "22"
})):
    print(1)
else:
    print(2)

2


In [ ]:
a class="b-stab-e-slider-item-e-txt js-search-result-tab"

In [204]:
re.sub(
    r'[^0-9]',
    '',
    soup_dt.find_all("a", {"class": "product-title-link"})[0].attrs['href']
)

'413353'

In [164]:
soup_data.find_all("div", {"class": "publisher"})[0].text.split(': ')[1].split(', ')[0]

'АСТ'

In [156]:
soup_data.find("div", {"class": "publisher"}).text.split(': ')[1].split(', ')[0]

'Fanzon'

In [136]:
re.findall(r'(.*)\n', '\nЦена\n864р\n')[2]

'864р'

In [125]:
re.search(r'\n(.*)\n', soup_data.find_all("div", {"class": "buying-price"})[0].text)

<re.Match object; span=(0, 6), match='\nЦена\n'>

In [63]:
authorsDict(soup_data)

{'Автор': 'Лю Цысинь',
 'Переводчик': 'Накамура Дмитрий',
 'Редактор': 'Горинова Н.'}

In [50]:
soup_data.find_all("div", {"class": "authors"})[1].text

'Переводчик: Накамура Дмитрий'

In [92]:
test_lst = []

In [38]:
soup_data.find_all('<div class="buying-price-old">')

[]

In [39]:
price_old = soup_data.select('div class=buying-price-old')

SelectorSyntaxError: Invalid character '=' position 9
  line 1:
div class=buying-price-old
         ^

In [10]:
price_old

[]

In [3]:
soup_data

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html prefix="ya: http://webmaster.yandex.ru/vocabularies/" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<script>
if(typeof console === 'undefined') {
	var console = {
		time: function (a,b) {},
		timeLog: function (a,b) {}
	};
}
if(!('time' in console)) {
	console.time = function (a,b) {};
}
if(!('timeLog' in console)) {
	console.timeLog = function (a,b) {};
}
console.time('cart', 'start');</script>
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://css.labirint.ru" rel="dns-prefetch"/><link href="https://js.labirint.ru" rel="dns-prefetch"/><link href="https://img.labirint.ru" rel="dns-prefetch"/><link href="https://img1.labirint.ru" rel="dns-prefetch"/><link href="https://img2.labirint.ru" rel="dn